# Program #2: Image Recognition on CIFAR-10 using Custom CNN and AlexNet

**Objective:** Design, implement, and compare two models for image classification on the CIFAR-10 dataset:
1.  A custom-built Convolutional Neural Network (CNN).
2.  An adapted version of the AlexNet architecture.

We will evaluate both models on their test accuracy and analyze their performance.

--- 
## 1. Setup and Data Preprocessing
First, we import the necessary libraries. TensorFlow and Keras will be used for building and training the models. The CIFAR-10 dataset is conveniently available directly through `tensorflow.keras.datasets`, so we can import it without needing to download it manually.

In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import numpy as np
import matplotlib.pyplot as plt

print("TensorFlow Version:", tf.__version__)

2025-07-25 06:54:16.105294: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-25 06:54:16.118805: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1753426456.132812    4975 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1753426456.136985    4975 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1753426456.150406    4975 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

TensorFlow Version: 2.19.0


### Load and Prepare the CIFAR-10 Dataset
We load the dataset, which is already split into training and testing sets. We then perform two key preprocessing steps:
1.  **Normalization:** Pixel values are scaled from the `[0, 255]` range to the `[0, 1]` range. This helps the model converge faster.
2.  **One-Hot Encoding:** The integer labels (0-9) are converted into a binary vector format (e.g., label `3` becomes `[0, 0, 0, 1, 0, 0, 0, 0, 0, 0]`). This is necessary for the `categorical_crossentropy` loss function.

In [2]:
# Load the dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Normalize pixel values to be between 0 and 1
x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

# One-hot encode the labels
y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# Define class names for CIFAR-10
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']

print("Training data shape:", x_train.shape)
print("Test data shape:", x_test.shape)
print("Training labels shape:", y_train.shape)
print("Test labels shape:", y_test.shape)

Exception: URL fetch failure on https://www.cs.toronto.edu/~kriz/cifar-10-python.tar.gz: None -- [SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: unable to get local issuer certificate (_ssl.c:1016)

--- 
## 2. Custom-Built CNN Model
Here, we design a simple but effective CNN. The architecture consists of two convolutional blocks followed by two fully connected layers.
-   **Conv Block 1:** `Conv2D` (32 filters) -> `BatchNormalization` -> `ReLU` -> `MaxPooling2D`
-   **Conv Block 2:** `Conv2D` (64 filters) -> `BatchNormalization` -> `ReLU` -> `MaxPooling2D`
-   **Classifier:** `Flatten` -> `Dense` (128 units, ReLU) -> `Dense` (10 units, Softmax)

In [ ]:
def build_custom_cnn(input_shape=(32, 32, 3)):
    model = Sequential([
        # First convolutional block
        Conv2D(32, (3, 3), padding='same', activation='relu', input_shape=input_shape),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        
        # Second convolutional block
        Conv2D(64, (3, 3), padding='same', activation='relu'),
        BatchNormalization(),
        MaxPooling2D(pool_size=(2, 2)),
        
        # Classifier head
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax') # 10 classes for CIFAR-10
    ])
    
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

custom_cnn_model = build_custom_cnn()
custom_cnn_model.summary()

### Train the Custom CNN

In [ ]:
print("\nTraining the Custom CNN Model...")
history_custom_cnn = custom_cnn_model.fit(
    x_train, y_train,
    epochs=20, 
    batch_size=64,
    validation_data=(x_test, y_test)
)

### Evaluate the Custom CNN

In [ ]:
loss_custom, acc_custom = custom_cnn_model.evaluate(x_test, y_test, verbose=0)
print(f"\nCustom CNN Test Accuracy: {acc_custom*100:.2f}%")

--- 
## 3. Adapted AlexNet Model
The original AlexNet was designed for 227x227 images. We must adapt it for CIFAR-10's 32x32 images. The key changes are:
-   Using smaller kernel sizes (e.g., `(3,3)` or `(5,5)`) and strides (`(1,1)`).
-   Reducing the number of filters and dense units to prevent overfitting on the smaller dataset.
-   Maintaining the core structure: multiple convolutional layers followed by max pooling, and a deep, dense classifier with `Dropout` for regularization.

In [ ]:
def build_adapted_alexnet(input_shape=(32, 32, 3)):
    model = Sequential([
        # Layer 1
        Conv2D(96, (3, 3), padding='same', activation='relu', input_shape=input_shape),
        MaxPooling2D(pool_size=(2, 2)),
        
        # Layer 2
        Conv2D(256, (3, 3), padding='same', activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        
        # Layer 3, 4, 5 (as in original AlexNet, but adapted)
        Conv2D(384, (3, 3), padding='same', activation='relu'),
        Conv2D(384, (3, 3), padding='same', activation='relu'),
        Conv2D(256, (3, 3), padding='same', activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        
        # Classifier
        Flatten(),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(4096, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

alexnet_model = build_adapted_alexnet()
alexnet_model.summary()

### Train the Adapted AlexNet

In [ ]:
print("\nTraining the Adapted AlexNet Model...")
history_alexnet = alexnet_model.fit(
    x_train, y_train,
    epochs=20,
    batch_size=64,
    validation_data=(x_test, y_test)
)

### Evaluate the Adapted AlexNet

In [ ]:
loss_alexnet, acc_alexnet = alexnet_model.evaluate(x_test, y_test, verbose=0)
print(f"\nAdapted AlexNet Test Accuracy: {acc_alexnet*100:.2f}%")

--- 
## 4. Comparative Analysis and Conclusion


### Performance Summary
Let's compare the final test accuracies of both models.

In [ ]:
print("--- Performance Comparison ---")
print(f"Custom CNN Test Accuracy:      {acc_custom*100:.2f}%")
print(f"Adapted AlexNet Test Accuracy: {acc_alexnet*100:.2f}%")

### Observations

*(Note: The exact accuracies will vary slightly with each training run due to random initializations.)*

Typically, the **Adapted AlexNet model performs better** than the simple, custom-built CNN. 

Here are the likely reasons for the performance difference:

1.  **Architectural Depth and Complexity:**
    -   **AlexNet:** It is a much deeper network with 5 convolutional layers and 3 dense layers. This depth allows it to learn a more complex and hierarchical representation of features, which is beneficial for a diverse dataset like CIFAR-10.
    -   **Custom CNN:** Our custom model is shallower (2 convolutional layers, 2 dense layers). While efficient, it has a lower learning capacity and may not capture the intricate details needed to differentiate between all 10 classes as effectively.

2.  **Number of Parameters (Model Capacity):**
    -   The adapted AlexNet has significantly more parameters than the custom CNN (tens of millions vs. hundreds of thousands). This higher capacity means it can learn more complex functions and patterns from the training data.

3.  **Regularization:**
    -   AlexNet famously introduced `Dropout` as a highly effective regularization technique. Our adapted version uses it in the dense layers, which helps prevent overfitting despite the large number of parameters. Our custom CNN also uses dropout, but AlexNet's deeper structure benefits more significantly from it.

### Conclusion

The adapted AlexNet architecture, due to its greater depth and capacity, is better suited to capture the complexity of the CIFAR-10 dataset, leading to higher test accuracy compared to the simpler custom CNN. This demonstrates the value of using deeper, well-established architectures as a starting point for complex image recognition tasks.